In [1]:
import pandas as pd
import numpy as np

click_columns = ['user_id','item_id','time']
train_click_T = pd.DataFrame()
test_click_T = pd.DataFrame()
test_qtime_T = pd.DataFrame()


# 第t个阶段，读取所有文件并合并
t = 4
for i in range(t):
    temp_train_click = pd.read_csv('../data/underexpose_train/underexpose_train_click-%s.csv'%(str(i)),sep=',',names = click_columns )
    temp_test_click = pd.read_csv('../data/underexpose_test/underexpose_test_click-%s.csv'%(str(i)),sep=',',names = click_columns)
    temp_test_qtime = pd.read_csv('../data/underexpose_test/underexpose_test_qtime-%s.csv'%(str(i)),sep=',',names = ['user_id','time'])
    train_click_T = train_click_T.append(temp_train_click)
    test_click_T = test_click_T.append(temp_test_click)
    test_qtime_T = test_qtime_T.append(temp_test_qtime)

In [2]:
test_qtime_T.head()

,user_id,time
0,11,0.983869
1,22,0.983956
2,44,0.983924
3,55,0.983953
4,66,0.983895


In [11]:
whole_dataset = pd.concat([train_click_T,test_click_T]).reset_index(drop=True)
whole_dataset

,user_id,item_id,time
0,4965,18,0.983763
1,20192,34,0.983772
2,30128,91,0.983780
3,29473,189,0.983930
4,10625,225,0.983925
...,...,...,...
1084190,34323,116409,0.983952
1084191,23433,116550,0.984105
1084192,11751,116908,0.984015
1084193,12565,117502,0.984052


In [23]:
import tqdm

test_semi = pd.DataFrame(columns=['user_id','pre2','pre1'])

for row in tqdm.tqdm(range(test_qtime_T.shape[0])):

    user_id = test_qtime_T.iloc[row,0]
    time = test_qtime_T.iloc[row,1]

    pre2 = whole_dataset[(whole_dataset['user_id'] == user_id) & (whole_dataset['time'] <= time)].iloc[-2,1]
    pre1 = whole_dataset[(whole_dataset['user_id'] == user_id) & (whole_dataset['time'] <= time)].iloc[-1,1]

    test_semi.loc[row,'user_id'] = user_id
    test_semi.loc[row,'pre2']    = pre2
    test_semi.loc[row,'pre1']    = pre1

100%|██████████| 6754/6754 [01:10<00:00, 96.07it/s]


In [24]:
test_semi.shape[0] == test_qtime_T.shape[0]

True

In [26]:
test_semi.to_csv('test-semi.csv',index=False)